<a href="https://colab.research.google.com/github/LeFalko/detect_CS/blob/master/Detect_CS_GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 1. Let Google access your Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 2. Install the toolbox on your Google Drive
import os

#download U'n'Eye
!git clone https://github.com/berenslab/uneye.git

# remove folders and files that are not needed
os.chdir('uneye')
import shutil
files = ['README.md','UnEye.ipynb','logo.jpeg','requirements_lin.txt','requirements_mac.txt','requirements_wind.txt','setup.py']
for i,f in enumerate(files):
    try:
        os.remove(f)
    except:
        print("File already deleted: ", f)
          
folders = ['.git','analysis scripts','data']
for i,f in enumerate(folders) :
    try:
        shutil.rmtree(f)
    except:
        print("Folder already deleted: ", f)
os.makedirs('training',exist_ok = True)
os.makedirs('work',exist_ok = True)
os.makedirs('work/LabelYourData',exist_ok = True)
os.makedirs('work/Output',exist_ok = True)
os.makedirs('work/TrainYourNetwork',exist_ok = True)

# add a patch to use U'n'Eye as CS detector
!git clone https://github.com/LeFalko/detect_CS.git

# to do math operations
import numpy as np
import scipy.io as io

# to read .mat files
!pip install mat4py
!pip install mat73
import mat4py
# to do read .pkl files
import pandas as pd

# to do clustering
!pip install hdbscan

import uneye
!pip install umap-learn
from detect_CS import *
#to download files from the internet
import urllib

# upload/download files
from google.colab import files


In [ ]:
#@title 3. Upload your training data
print('Upload your file here')
uploaded = files.upload()
filename = next(iter(uploaded))
print('your file \'' + filename + '\' was uploaded')

In [ ]:
#@title 4. specify name of weights you want to save
# the weights will be saved in the folder 'training'
weights_name = 'my_weights' #@param {type:"string"}


In [ ]:
#@title Advanced setting 
#@markdown You don't need to change this, just run this cell.

#@markdown ##### kernel size (default=9), needs to be odd
kernel_size = 9 #@param {type:"slider", min:1, max:15, step:2}
#@markdown ##### max pooling (default=7), needs to be odd
max_pooling = 7 #@param {type:"slider", min:1, max:15, step:2}
#@markdown ##### If you have a small training set you should consider reducing this number (default=10). Otherwise the training will show an error
val_samples = 10 #@param {type:"slider", min:1, max:20, step:1}

In [ ]:
#@title 5. Train your network
DATA = io.loadmat(filename)

ID = DATA['ID']
bigTraining_LFP = DATA['LFP'].squeeze()
bigTraining_HIGH = DATA['HIGH'].squeeze()
bigTraining_LABEL = DATA['Labels'].squeeze()

def squizz(data):
    data = [x.squeeze() for x in data]
    return(data)
bigTraining_LFP = squizz(bigTraining_LFP)
bigTraining_HIGH = squizz(bigTraining_HIGH)
bigTraining_LABEL = squizz(bigTraining_LABEL)
DATA = None

sampfreq = 25000 # Sampling frequency of your signal in Hz

length = int(sampfreq*1) # we will use only 1 s of data from each cell
trainingLFP = []
trainingHIGH = []
trainingLabel = []
for ii,txt in enumerate(ID):
  trainingLFP.append(bigTraining_LFP[ii][:length])
  trainingHIGH.append(bigTraining_HIGH[ii][:length])
  trainingLabel.append(bigTraining_LABEL[ii][:length])

trainingLFP = np.concatenate(trainingLFP)  
trainingHIGH = np.concatenate(trainingHIGH)  
trainingLabel = np.concatenate(trainingLabel) 

# size of the convolutional and maxpooling operations
# ks = 9 # convolution kernel size. NEED TO BE ODD.
# mp = 7 # maxpooling size. NEED TO BE ODD.

# the number of bins (nb) taken into account by the network is given by
# nb = mp**2+mp**2*ks+(mp*ks)-mp+2*ks-2 

# val_samples = 10 # If you have a small training set you should consider reducing this number.
                 # Otherwise the training will show an error
#########################
model = uneye.DNN(max_iter = 3000, ks=kernel_size,mp=max_pooling, weights_name=weights_name,
                  sampfreq = sampfreq,augmentation = False,val_samples = val_samples, doDiff = False)

model.train(trainingLFP,trainingHIGH,trainingLabel)

In [ ]:
#@title 6. download your weights 
download_name = './training/' + weights_name
files.download(download_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>